3.4 chaining chains - create language teacher chain



In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.9, 
    streaming = True, 
    callbacks=[StreamingStdOutCallbackHandler()]) # StreamingStdOutCallbackHandler - print out every available letter

langTeach_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Ukrainian teacher who teaches Ukrainian Language. You create easy examples and explanations. And you only reply in Ukrainian."),
    ("human", "Please explain about {anything} in {language1}")
])

langTeach_chain = langTeach_prompt | chat

3.4 chaining chains  - create Korean tutor chain
 

In [12]:
korTutor_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Korean tutor who helps Korean students learning a foreign language. You usually translate explanations in Korean but you can modify the example and make alternative explanation which is more familiar and easy to understand for Korean students. And you usually reply in Korean, use other language only if it is necessary"),
    ("human", " {previous_answer} ")
])

kor_chain = korTutor_prompt | chat


final_chain = {"previous_answer":langTeach_chain} | kor_chain

final_chain.invoke({
    "language1": "Ukrainian",
    "anything" : "How can I say You eat so small"
})


Ви їсте так мало. 

Українське речення складається із займенника "Ви" (You), дієслова "їсте" (eat) і прислівника "так" (so), який вказує на ступінь порівняння, та прикметника "мало" (small), який вказує на кількісну ознаку."Ви їсте так мало."라는 문장은 "당신은 너무 적게 먹어요."로 해석됩니다. 이 문장은 "Ви" (당신), "їсте" (먹다) 동사, "так" (너무) 비교급을 나타내는 부사, "мало" (적은) 양을 나타내는 형용사로 구성되어 있습니다.

AIMessage(content='"Ви їсте так мало."라는 문장은 "당신은 너무 적게 먹어요."로 해석됩니다. 이 문장은 "Ви" (당신), "їсте" (먹다) 동사, "так" (너무) 비교급을 나타내는 부사, "мало" (적은) 양을 나타내는 형용사로 구성되어 있습니다.', response_metadata={'finish_reason': 'stop'}, id='run-6cdd7e51-a61e-40f0-9d9d-07ab3f8c19a1-0')

4.1 - Fewshot template - show the models some examples to make the answer better
4.2 - FewshotChatMessage template
4.3 - Example selectors - LengthBasedExampleSelector, BaseExampleSelector


In [45]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate, ChatPromptTemplate

from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature = 0.9, 
    streaming = True, 
    callbacks=[StreamingStdOutCallbackHandler()]
    ) 

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# examples = [
#     {
#         "city": "Kyoto",
#         "answer": """
#         Here is what I recommend:
#         Place: Fushimi Inari Taisha, Arashiyama Bamboo forest, Kiyomizudera
#         Food: Matcha Green tea and Tenkaippin Ramen
#         To do : Wear kimono and walk around the shirine, Try Traditional Tea ceremony 
#         """,
#     },
#     {
#         "city": "Tokyo",
#         "answer": """
#          Here is what I recommend:
#         Place: Senso-ji Temple, Shibuya Crossing, Meiji Shrine
#         Food: Sushi, Ramen, Matcha Desserts
#     T   To Do: Explore the historic Asakusa district in traditional Yukata, participate in a sushi-making class, visit  teamLab Borderless for a digital art experience
#         """,
#     },
#     {
#         "city": "Vancouver",
#         "answer": """
#         Here is what I recommend:
#         Place: Stanley Park, Granville Island, Capilano Suspension Bridge
#         Food: Pacific Northwest Seafood, Japadog, Poutine
#         To Do: Bike around Stanley Park, explore the public market on Granville Island, walk through the Capilano Suspension Bridge Park
#         """,
#     },
#         {
#         "city": "Calgary",
#         "answer": """
#         Here is what I recommend:
#         Place: Calgary Tower, Heritage Park, Banff National Park (nearby)
#         Food: Alberta Beef, Game Meat, Canadian Rye Whiskey
#         To Do: Visit the Calgary Tower for panoramic city views, explore Canadian history at Heritage Park, take a day trip to Banford National Park for hiking or skiing
#         """,
#     },
# ]

example_template = """
    Human:{question}
    AI: {answer}

"""

example_prompt = PromptTemplate.from_template(example_template)

# example_prompt = ChatPromptTemplate.from_messages([
#     ("human", "What do you know about {city}"),
#     ("ai", "{answer}")
# ])

class RandomExampleSelector(BaseExampleSelector): #customed example selector - can be used like: logined user can give more examples

    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)
   
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)] #return one example randomly

   

# example_selector = LengthBasedExampleSelector(
#     examples=examples,
#     example_prompt=example_prompt,
#     max_length= 280,
# )

example_selector  = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
 #   examples= examples, #format all examples
    example_selector=example_selector,
    suffix= "Human: What do you know about {country}?",
    input_variables=["country"]
    )

# prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=example_prompt,
#     examples=examples
# )
prompt.format(country = "Korea")

# final_prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a Seoul Travel Guide."),
#     prompt,
#     ("human", "What do you know about {city}")
# ])



# chain = prompt | chat
# chain =  final_prompt | chat
# chain.invoke({
#     "country": "Ukraine"
# })

# chain.invoke({
#     "city": "Seoul"
# })

'\n    Human:What do you know about France?\n    AI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\n\nHuman: What do you know about Korea?'

It is a bit chatty then examples.

The reason can be it is the chat model, or system message "You are a Seoul Travel Guide",
and also  temperature is high.
"You give short answers" can fix this.